<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Supmodel_v2/M_maxlen7_350_icd/CNNmodelv2_v4.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Builf train and evaluate
# Save model for offline scoring on test
# Model name: CCP_ICD10_Searchengine/CNNmodel_v1.0.ipynb
# References
#https://github.com/njainds/Colab_notebooks/blob/master/Kaggle_Toxic_Comments/Model-2-keras_lstmConv.ipynb
#https://github.com/njainds/NLP/blob/master/kaggle/QIQC/22nd%20Place%20Solution%20-%206%20models%20%2B%20POS%20tagging.ipynb
#https://github.com/Cheneng/DPCNN/blob/master/model/DPCNN.py


In [0]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [0]:

import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')

import os
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import Input, Dense, Embedding, concatenate, CuDNNGRU, CuDNNLSTM, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, Flatten, Lambda, Permute, Reshape, merge, Dropout, Conv2D, MaxPool2D, Concatenate, Conv1D, MaxPool1D, add, MaxPooling1D
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D, Add
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam, RMSprop
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import callbacks

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,linear_model,metrics
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer
from gensim.models.keyedvectors import KeyedVectors

In [2]:
from google.colab import drive
drive.mount('/content/drive')
#!ls "/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2"
os.chdir("/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2/d_maxlen7_350_icd")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
x_train = np.load("train_X.npy")
x_test = np.load("test_X.npy")
y_train = np.load("train_y.npy")
y_test = np.load("test_y.npy")
features = np.load("test_y.npy")
icd_dict = np.load("icd_dict.npy").item()
word_index = np.load("word_index.npy").item()
embeddings = np.load("embedding_matrix.npy")

In [0]:
class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=4590., mode='triangular',gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
    
        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size,
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
            
    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        K.set_value(self.model.optimizer.lr, self.clr())

In [5]:
class RocAucEvaluation(keras.callbacks.Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

#del check_point, ra_val, early_stop
!rm ./best_model.hdf5
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(x_test, y_test), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

rm: cannot remove './best_model.hdf5': No such file or directory


In [6]:
#configs
os.chdir("/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2/M_maxlen7_350_icd")
max_features = embeddings.shape[0]
maxlen = x_train.shape[1]
embed_size = embeddings.shape[1]
n_class = y_train.shape[1]
print(max_features,maxlen, embed_size, n_class)

13735 7 200 350


In [0]:
def build_dpcnn_model(num_block = 5, k = 3, units = 128):
    pad = 'same'
    keras.backend.clear_session()
    inp = Input(shape=(maxlen,))
    embedding_layer1   = Embedding(max_features, embed_size, weights=[embeddings], trainable=True)(inp)
    embedding_layer1 = SpatialDropout1D(0.2)(embedding_layer1)
    emb_short_cut = Conv1D(units, kernel_size = 1, padding = pad, activation = 'relu')(embedding_layer1)
    emb_short_cut = Conv1D(units, kernel_size = 1, padding = pad, activation = 'relu')(emb_short_cut)
    # Main block
    for b in range(1, num_block + 1):
        if b == 1:
            block = embedding_layer1
            short_cut = emb_short_cut
        else:
            block = block
            short_cut = block
        block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(block)
        block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(block)
        block = add([short_cut, block])
        block = MaxPooling1D(pool_size = 3, strides = 2, padding = pad)(block)
    # Final block
    short_cut = block
    block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(block)
    block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(block)
    block = add([short_cut, block])
    max_pool = GlobalMaxPooling1D()(block)
    avg_pool = GlobalAveragePooling1D()(block)
    last = (Lambda(lambda x: x[:,-1,:]) (block))
    block = concatenate([max_pool, avg_pool, last])
    #reverse
    rev_embedding_layer = Lambda(lambda x: K.reverse(x,axes=-1))(embedding_layer1)
    rev_emb_short_cut = Conv1D(units, kernel_size = 1, padding = pad, activation = 'relu')(rev_embedding_layer)
    rev_emb_short_cut = Conv1D(units, kernel_size = 1, padding = pad, activation = 'relu')(rev_emb_short_cut)
    # Main block
    for b in range(1, num_block + 1):
        if b == 1:
            rev_block = rev_embedding_layer
            rev_short_cut = rev_emb_short_cut
        else:
            rev_block = rev_block
            rev_short_cut = rev_block
        rev_block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(rev_block)
        rev_block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(rev_block)
        rev_block = add([rev_short_cut, rev_block])
        rev_block = MaxPooling1D(pool_size = 3, strides = 2, padding = pad)(rev_block)
    # Final block
    rev_short_cut = rev_block
    rev_block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(rev_block)
    rev_block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(rev_block)
    rev_block = add([rev_short_cut, rev_block])
    rev_max_pool = GlobalMaxPooling1D()(rev_block)
    rev_avg_pool = GlobalAveragePooling1D()(rev_block)
    rev_last = (Lambda(lambda x: x[:,-1,:]) (rev_block))
    rev_block = concatenate([rev_max_pool, rev_avg_pool, rev_last])
    block = concatenate([rev_block, block])
    # output block
    out_put = Dense(128, activation = 'relu')(block)
    outp = Dense(n_class, activation='softmax')(out_put)
    model=Model(inputs=inp,outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=["accuracy"])
    return model

In [34]:
# Kfold training of model

kfold = StratifiedKFold(n_splits=5, random_state=2019, shuffle=True)
pred_test = np.zeros((x_test.shape[0], n_class))

for i, (train_index, valid_index) in enumerate(kfold.split(x_train, np.argmax(y_train, axis=1))):
    X_train, X_val, Y_train, Y_val = x_train[train_index], x_train[valid_index], y_train[train_index], y_train[valid_index]
    
    file_path = "best_model.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
    ra_val = RocAucEvaluation(validation_data=(X_val, Y_val), interval = 1)
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
    
    model = build_dpcnn_model(num_block = 5, k = 3, units = 64)
    
    print("### Model with seed: {}  for fold no. {} ####".format( 2019, i))
    
    clr = CyclicLR(base_lr=1e-6, max_lr=5e-4,step_size=10840., mode='exp_range',gamma=0.99994)
    model.fit(X_train, Y_train, batch_size = 24, epochs = 10, validation_data = (X_val, Y_val), verbose = 1, callbacks = [clr, ra_val, check_point, early_stop])
    
    pred_train = model.predict([X_val], batch_size=24, verbose=2)
    pred_test += model.predict([x_test], batch_size=16, verbose=1)/5
    os.remove(file_path)



### Model with seed: 2019  for fold no. 0 ####
Train on 41496 samples, validate on 10537 samples
Epoch 1/10
41496/41496 [==============================] - 56s 1ms/step - loss: 5.2488 - acc: 0.0875 - val_loss: 4.3082 - val_acc: 0.2142

 ROC-AUC - epoch: 1 - score: 0.781899

Epoch 00001: val_loss improved from inf to 4.30825, saving model to best_model.hdf5
Epoch 2/10
41496/41496 [==============================] - 54s 1ms/step - loss: 3.4839 - acc: 0.3247 - val_loss: 2.7180 - val_acc: 0.4367

 ROC-AUC - epoch: 2 - score: 0.940334

Epoch 00002: val_loss improved from 4.30825 to 2.71805, saving model to best_model.hdf5
Epoch 3/10
41496/41496 [==============================] - 53s 1ms/step - loss: 2.4516 - acc: 0.4768 - val_loss: 2.1317 - val_acc: 0.5433

 ROC-AUC - epoch: 3 - score: 0.967644

Epoch 00003: val_loss improved from 2.71805 to 2.13165, saving model to best_model.hdf5
Epoch 4/10
41496/41496 [==============================] - 53s 1ms/step - loss: 1.9657 - acc: 0.5661 - val_loss: 

In [0]:
pred_test=pred_test*8/7

In [0]:
# ROC scores on Test set for different codes
scores=[]
for i in range(396):
  score = roc_auc_score(y_test[:,i], pred_test[:,i])
  scores.append(score)
  #print("score for index %s is %d" (i,score))

low = list(np.argsort(scores)[:5])
high = list(np.argsort(scores)[::-1][:250])
print('high scores is {}'.format(np.mean([scores[i] for i in high])))
print('low scores is {}'.format(np.mean([scores[i] for i in low])))
# ROC is not a problem in multi-class prediction

high scores is 0.9976635485766142
low scores is 0.8496196545022776


In [0]:
# Saving predictions for error analysis
os.mkdir('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2/M_maxlen7_350_icd/run2_embed_dpcnn_73.1/')
os.chdir('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2/M_maxlen7_350_icd/run2_embed_dpcnn_73.1')

itoicd = dict((v,k) for k,v in icd_dict.items())
test_pred = np.argmax(pred_test, axis=1)
test_act = np.argmax(y_test, axis=1) 
mdf = pd.DataFrame({'test_act':[itoicd[i] for i in list(test_act)],'test_pred':[itoicd[i] for i in list(test_pred)]})
#mdf.to_csv('pred_test.csv',index=None)

In [0]:
# Top Codes with low accuracy on test set
#pred_test
acc = [len(mdf[(mdf['test_pred']==i) & (mdf['test_act']==i)])/len(mdf[mdf['test_act']==i]) for i in mdf['test_act'].values]
mdf2 = pd.DataFrame({'class_index': mdf['test_act'].values,'accuracy':acc})
#print(len(mdf2[mdf2['accuracy']<0.5].class_index.unique().tolist()))
codes= mdf2[mdf2['accuracy']<0.5].class_index.unique().tolist()
#mdf2.to_csv('error_codes.csv', index=None)

In [37]:
# Accuracy on various condifence levels and sample coverage on Test set
test_pred = np.argmax(pred_test, axis=1)
test_pred_proba = np.max(pred_test,axis=1)
mdf3 = pd.DataFrame({'test_act':list(test_act),'test_pred':list(test_pred), 'prob': list(test_pred_proba)})
#del CUTOFF
for CUTOFF in range(0,100,5):
    acc = len(mdf3[(mdf3['prob']>CUTOFF/100) & (mdf3['test_act']==mdf3['test_pred'])])/len(mdf3[(mdf3['prob']>CUTOFF/100)])
    cov = len(mdf3[(mdf3['prob']>CUTOFF/100)])/len(mdf3)
    print("Confidence: %f ## Accuracy: %f ## Coverage on Test dataset: %f" % (CUTOFF/100,acc,cov))
    #print("coverage with this accuracy level on Test dataset is {}".format(cov))


  

Confidence: 0.000000 ## Accuracy: 0.731581 ## Coverage on Test dataset: 1.000000
Confidence: 0.050000 ## Accuracy: 0.747259 ## Coverage on Test dataset: 0.977862
Confidence: 0.100000 ## Accuracy: 0.757777 ## Coverage on Test dataset: 0.961778
Confidence: 0.150000 ## Accuracy: 0.769683 ## Coverage on Test dataset: 0.942407
Confidence: 0.200000 ## Accuracy: 0.780479 ## Coverage on Test dataset: 0.924939
Confidence: 0.250000 ## Accuracy: 0.794186 ## Coverage on Test dataset: 0.898305
Confidence: 0.300000 ## Accuracy: 0.807014 ## Coverage on Test dataset: 0.872881
Confidence: 0.350000 ## Accuracy: 0.820539 ## Coverage on Test dataset: 0.847112
Confidence: 0.400000 ## Accuracy: 0.833122 ## Coverage on Test dataset: 0.818748
Confidence: 0.450000 ## Accuracy: 0.851737 ## Coverage on Test dataset: 0.781563
Confidence: 0.500000 ## Accuracy: 0.864827 ## Coverage on Test dataset: 0.745936
Confidence: 0.550000 ## Accuracy: 0.877675 ## Coverage on Test dataset: 0.711173
Confidence: 0.600000 ## Accu

In [0]:
# Steps- for re generating the Data for modeling

#datasize for icd codes

strain = np.argmax(np.vstack([y_train,y_test]), axis=1)
codesize = [sum(strain==icd_dict[i]) for i in codes]
#print(len([i for i in codesize if i < 30]))
codesize2 = [sum(strain==icd_dict[i]) for i in (set([itoicd[i] for i in set(strain)])-set(codes))]
#print(len([i for i in codesize2 if i < 30]))

print("mean dataset size of error codes is %d" % (np.mean(codesize)))
print("mean  dataset size of non-error codes is %d" % (np.mean(codesize2)))
print("median dataset size of error codes is %d" % (np.median(codesize)))
print("median  dataset size of non-error codes is %d" % (np.median(codesize2)))

# word entropy for icd codes

idx = [[j for j in range(len(strain)) if strain[j]==icd_dict[i]] for i in codes]
idx2 = [[j for j in range(len(strain)) if strain[j]==icd_dict[i]] for i in (set([itoicd[i] for i in set(strain)])-set(codes))]
x = np.vstack([x_train, x_test])
word_ent = [len(set([j for i in [[list(x[i]) for i in p] for p in idx][p] for j in i])) for p in range(len([[list(x[i]) for i in p] for p in idx]))]
word_ent2 = [len(set([j for i in [[list(x[i]) for i in p] for p in idx2][p] for j in i])) for p in range(len([[list(x[i]) for i in p] for p in idx2]))]

print("#################################################")
print("mean # of unqiue words in error codes is %d" % (np.mean(word_ent)))
print("mean # of unqiue words in non-error codes is %d" % (np.mean(word_ent2)))
print("median # of unqiue words in error codes is %d" % (np.median(word_ent)))
print("median # of unqiue words in non-error codes is %d" % (np.median(word_ent2)))


#codesize


mean dataset size of error codes is 70
mean  dataset size of non-error codes is 193
median dataset size of error codes is 50
median  dataset size of non-error codes is 104
#################################################
mean # of unqiue words in error codes is 76
mean # of unqiue words in non-error codes is 109
median # of unqiue words in error codes is 58
median # of unqiue words in non-error codes is 80


In [0]:
# Saving Model
# serialize model to JSON
model_json = model.to_json()
with open("model_v1.0.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_v1.0.h5")
print("Saved model to disk")
 
# later...

Saved model to disk


In [0]:
# load json and create model
json_file = open('model_v1.0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_v1.0.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-3), metrics=["accuracy"])
score = np.argmax(loaded_model.predict(DATASET, batch_size = 16, verbose = 1), axis=1)